In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
sys.path.append("../src")

In [3]:
import numpy as np
from pydrake.all import (
    StartMeshcat,
    Simulator
)

In [4]:
from world import make_n_quadrotor_system
from util import DisableCollisionChecking
from stabilization import find_fixed_point_snopt, lqr_stabilize_to_point, add_controller_to_system

In [5]:
meshcat = StartMeshcat()

INFO:drake:Meshcat listening for connections at http://localhost:7000


In [6]:
np.random.seed(0)

cable_length = 2
cable_hooke_K = 10
free_body_mass = 1
n_quadrotors = 4
quadrotor_input_limit = 3 # or None

min_quadrotor_distance = 1 # or None
min_cable_length = 2.1
controller_time_horizon = 10

In [7]:
diagram, plant = make_n_quadrotor_system(meshcat,
                                         n_quadrotors,
                                         cable_length,
                                         cable_hooke_K,
                                         free_body_mass)

In [8]:
fixed_point, fixed_control = find_fixed_point_snopt(diagram,
                                                    limit=quadrotor_input_limit,
                                                    min_quadrotor_distance=min_quadrotor_distance,
                                                    min_cable_length=min_cable_length)
print(fixed_point)
print(fixed_control)

Failed
True
[-6.07121903e-01  6.08870784e-01  3.77426910e-01 -1.76863166e-01
  3.05978943e+00  5.64243183e-02 -3.55496065e-01 -3.54251001e-01
  4.72720475e-01 -1.39585252e-07 -3.93415299e-02  3.91750401e-02
  6.06731211e-01 -6.09265854e-01  3.77399465e-01  1.76859024e-01
  5.63246349e-02 -3.05972235e+00  3.52764479e-01  3.51518615e-01
  4.88728275e-01  6.81662747e-08  3.96069677e-02 -3.94348006e-02
 -4.24261300e-07 -3.33286346e-07  6.38220532e-07 -6.87408427e-07
  3.12211810e-03  3.12441425e-03 -1.71627484e+00  0.00000000e+00
  4.34140421e-20 -5.58586321e-18  2.46583058e-13 -4.97645975e-16
  4.01652029e-17  8.83519295e-18 -1.22076036e-17  2.50372871e-18
  5.80949830e-18  1.07145477e-17 -1.18508269e-18  0.00000000e+00
 -1.33357869e-18 -3.42502914e-18  0.00000000e+00 -2.77555756e-17
 -3.12407971e-18  3.90107381e-19 -3.40436896e-18 -1.14754706e-19
  2.90589796e-20  0.00000000e+00 -2.82777063e-18 -1.71895961e-06
 -2.12218015e-06 -4.42278803e-06 -9.13425211e-18  2.86743465e-18
 -1.73713783e

In [9]:
# Check that it's a fixed point

simulator = Simulator(diagram)
simulator.set_target_realtime_rate(1)
context = simulator.get_mutable_context()

sg = diagram.GetSubsystemByName("scene_graph")
DisableCollisionChecking(sg, context)

u = diagram.GetInputPort("u")
u.FixValue(context, fixed_control)

eps = 0

# Simulate
context.SetTime(0.0)
context.SetContinuousState(
    fixed_point + np.random.normal(loc=0, scale=eps, size=fixed_point.shape)
)

simulator.set_publish_every_time_step(True)
simulator.Initialize()
simulator.AdvanceTo(5.0)
print(simulator.get_actual_realtime_rate())

1.0019534690011542


In [10]:
# Make an LQR controller

Q_quadrotor_pos = [10.] * 6
Q_quadrotor_vel = [1.] * 6
Q_freebody_pos = [1.] * 4 + [10.] * 3
Q_freebody_vel = [1.] * 3 + [1.] * 3
Q_pos = Q_quadrotor_pos * n_quadrotors + Q_freebody_pos
Q_vel = Q_quadrotor_vel * n_quadrotors + Q_freebody_vel
Q = np.diag(Q_pos + Q_vel)
R = np.eye(4 * n_quadrotors)

lqr_controller = lqr_stabilize_to_point(diagram, fixed_point, fixed_control, Q, R, controller_time_horizon)

controlled_diagram, controlled_plant = add_controller_to_system(diagram, lqr_controller, limit=quadrotor_input_limit)

In [12]:
# Simulate the LQR controller

simulator = Simulator(controlled_diagram)
simulator.set_publish_every_time_step(True)
simulator.set_target_realtime_rate(1.0)
context = simulator.get_mutable_context()

sg = diagram.GetSubsystemByName("scene_graph")
DisableCollisionChecking(sg, context)

pos_noise = 0.5
orien_noise = 0.25
vel_noise = 0.1

vel_mask = np.array(
    [False] * (6 * n_quadrotors + 7) +
    [True] * (6 * n_quadrotors + 6)
)
pos_mask = np.array(
    ([True] * 3 + [False] * 3) * n_quadrotors +
    [False] * 4 + [True] * 3 + 
    ([True] * 3 + [False] * 3) * n_quadrotors +
    [False] * 3 + [True] * 3
)
orien_mask = np.logical_not(pos_mask)

pos_mask = np.logical_and(pos_mask, np.logical_not(vel_mask))
orien_mask = np.logical_and(orien_mask, np.logical_not(vel_mask))

# Simulate
while True:
    init = fixed_point.copy()
    init[pos_mask] += np.random.normal(loc=0, scale=pos_noise, size=init[pos_mask].shape)
    init[orien_mask] += np.random.normal(loc=0, scale=orien_noise, size=init[orien_mask].shape)
    init[vel_mask] += np.random.normal(loc=0, scale=vel_noise, size=init[vel_mask].shape)
    context.SetTime(0.0)
    context.SetContinuousState(
        init
    )
    simulator.Initialize()
    simulator.AdvanceTo(controller_time_horizon)
    print(simulator.get_actual_realtime_rate())

0.9989057227544598
0.9989323633662259
0.9992270268496901
0.9993166507854682
0.9988922580530301
0.9995512947268035
0.9987309869403534
0.9993447093873571
0.9990749004017087
0.9986687214651111
0.99866794912622
0.9986524027688609
0.9991773929394779
0.9989060984310612
0.9989867938279461
0.999839491567093
0.9997906885204676
0.9984926844300187
0.9988728442138114
0.9992427986882998


KeyboardInterrupt: 